In [46]:
!pip install catboost
!pip install scikit-learn
!pip install -r requirements.txt

import pandas as pd
from catboost import CatBoostClassifier, cv
from sklearn.model_selection import train_test_split

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 6.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.3/383.3 k

In [49]:
from langchain_core.messages import BaseMessage 
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, AzureChatOpenAI
import openai

In [52]:
class NewsClassifier:
    def __init__(self, number_of_tags: int = 1):
        self.classifier = CatBoostClassifier()
        self.classifier.load_model('makar/model.txt')
        
        base_url = "http://saiga.vkcloud.eazify.net:8000/v1" # Saiga 3
        # base_url = "http://shwars.vkcloud.eazify.net:8000/v1" # Saiga Gemma
        # base_url = "http://10.0.0.242/v1" # Self hosted
        base_url = "http://hackllm.vkcloud.eazify.net:8000/v1" # Mail.ru

        self.chat = ChatOpenAI(api_key="123",
            model="/home/ubuntu/.cache/huggingface/hub/models--TheBloke--saiga_mistral_7b-GGUF/snapshots/7fb350fb3566d722ea526fd99d1ae9600fd4f6bc/saiga_mistral_7b.Q4_K_M.gguf",
            openai_api_base=base_url)

        assert 0 < number_of_tags <= 11
        self.number_of_tags = number_of_tags
    
    def predict(self, text: str) -> str:
        messages = [
            SystemMessage(content="Ты - анализатор статей"),
            HumanMessage(content=f"""
Прочитай приведённю ниже в тройных обратных кавычках статью и напиши ровно {self.number_of_tags} слов, которые
характеризуют данную статью, разделив слова запятыми.
Статья: ```{text}```"
""")
        ]
        try:
            return self.chat.invoke(messages).content.strip(" \n\r,;.").lower()
        except openai.UnprocessableEntityError:
            return self.chat.invoke(messages[:len(messages) // 2]).content.strip(" \n\r,;.").lower()

    def predicts(self, texts: list[str]) -> list[list[str]]:
        hmessage = HumanMessage(content="")
        messages = [
            SystemMessage(content="Ты - анализатор статей"),
            hmessage
        ]
        tags = []
        for text in texts:
            hmessage.content = f"""
Прочитай приведённю ниже в тройных обратных кавычках статью и напиши ровно {self.number_of_tags} слов, которые
характеризуют данную статью, разделив слова запятыми.
Статья: ```{text}```"
"""
            try:
                res = self.chat.invoke(messages).content.strip(" \n\r,;.").lower()
            except openai.UnprocessableEntityError:
                res = self.chat.invoke(messages[:len(messages) // 2]).content.strip(" \n\r,;.").lower()
            tags.append(res)
        return tags
    
    def get_category(self, name, tags):
        return self.classifier.predict([name, tags])
    
    def get_categories(self, df):
        cats = []
        for i in range(len(df)):
            cats.append(self.classifier.predict([df[i][0], df[i][1]]))
        return cats
    
    def scores(X_test, y_test):
        print(f"Scores")
        from sklearn.metrics import f1_score
        from sklearn.metrics import roc_auc_score
        from sklearn.preprocessing import LabelBinarizer

        # Преобразование меток классов в бинарные формы (нужно для roc_auc_score)
        lb = LabelBinarizer()
        lb.fit(y_test)
        y_test_bin = lb.transform(y_test)

        # Вычисление ROC-AUC score
        roc_auc = roc_auc_score(y_test_bin, lb.transform(news.get_categories(X_test)), average='macro')
        print(f"ROC-AUC score: {roc_auc}")

        print(f"F1-score: {f1_score(news.get_categories(X_test), y_test, average = 'micro')}")


In [53]:
news = NewsClassifier()

In [54]:
news.get_category('Россияне вдвое сократили интерес к жилью в Турции, но сохранили лидерство', 'Недвижимость в Турции')

array(['Здоровье'], dtype=object)

In [55]:
def scores(X_test, y_test):
    print(f"Scores")
    from sklearn.metrics import f1_score
    from sklearn.metrics import roc_auc_score
    from sklearn.preprocessing import LabelBinarizer

    # Преобразование меток классов в бинарные формы (нужно для roc_auc_score)
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_bin = lb.transform(y_test)

    # Вычисление ROC-AUC score
    roc_auc = roc_auc_score(y_test_bin, lb.transform(news.get_categories(X_test)), average='macro')
    print(f"ROC-AUC score: {roc_auc}")

    print(f"F1-score: {f1_score(news.get_categories(X_test), y_test, average = 'micro')}")


In [56]:
df = pd.read_csv('track1.csv')

In [57]:
scores(df[0], df[1])

KeyError: 0